# Introduction
blabla

### Importation des modules nécessaires
On charge les modules essentiels pour le fonctionnement du code

In [ ]:
import random
import numbers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Fonctions pour gérer matrices de distance
- Matrices aleatoires
- Matrices de coordonnées TSP

In [ ]:

# Fonction pour générer matrices de distance aléatoires
# Entrées: - n_cities : nombre de villes dans la réseau
#          - min_dist : distance minimale entre deux villes
#          - max_dist : distance maximale entre deux villes
#          - export : true si l'on veux exporter la matrice dans un fichier
#          - file : nom du fichier de sortie
# Sorties: - matrix : matrice de distances
def generate_map(n_cities = 10, min_dist = 10, max_dist = 100, export = True, file = "matrix.csv"):
    matrix = np.zeros((n_cities, n_cities), dtype=int)

    # Pour chaque ville, on choisit une distance avec les autres villes
    for i in range(n_cities):
        # On assume distances symétriques, donc on parcours juste le triangle supérieur
        for j in range(i + 1, n_cities):
            dist = random.randint(min_dist, max_dist)
            matrix[i][j] = dist
            matrix[j][i] = dist

    # Si l'on veut exporter la matrice, on utilise pandas pour l'écrire dans un fichier csv
    # Si non, on retourne la matrice
    if export:
        df = pd.DataFrame(matrix)
        city_labels = [f'City{i}' for i in range(len(matrix))]
        df.index = city_labels
        df.columns = city_labels
        df.to_csv(file)
    else:
        return matrix



# Fonction pour importer des réseaux de villes en format tsp. Les fichiers sont
# la propriété intellectuelle de l'Université de Heidelberg :
# http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/
# Entrées: - filename : nom de fichier a importer
# Sorties: - dist_matrix : matrice de distances entre villes
#          - coords : coordonnées des villes
def import_tsplib(filename):
    coords = []
    
    # On vérifie chaque ligne du fichier. Si commence par un nombre,
    # on importe la coordonnée de la ville i.
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            parts = line.split()
            
            if isinstance(parts[0], numbers.Number):
                coords.append([float(parts[1]), float(parts[2])])
            elif line == "EOF":
                break

    coords = np.array(coords)
    dist_matrix = np.zeros((len(coords), len(coords)))
    
    # On calcule la distance euclidienne entre deux villes 
    # et remplit la matrice des distances.
    for i in range(len(coords)):
        for j in range(len(coords)):
            if i != j:
                dist_matrix[i,j] = np.linalg.norm(coords[i,:] - coords[j,:])
                
    return dist_matrix, coords

### Classes pour l'implementation de l'algorithme
bla

In [ ]:
# Classe pour représenter le réseau de villes
# Membres: - matrix : matrice de distance entre villes
#          - cities : liste des objets type Ville dans la réseau
#          - first : indice de la première ville
#          - current : indice de la ville actuelle pendant l'algorithme
#          - visit_order : liste d'indices des villes parcourues
#          - total_distance : distance totale parcouru
#          - visited : liste de booleans pour marquer les villes déjà visitées
#
# Méthodes: - add_visited_city(index) : ajoute un ville visité dans la liste visit_ordre
#                                       et change l'indice actuel
class Map():
	def __init__(self, file = None, n_cities = 10, min_dist = 10, max_dist = 100, index_first = 0):
		# On vérifie si un fichier à été donné. Si oui, on importe le fichier.
  		# Si non, on genere un matrice de distance aléatoire.
		if file == None:
			self.matrix = generate_map(n_cities, min_dist, max_dist, export = False)
		else:
			if file.split('.')[-1] == "csv":
				self.matrix = pd.read_csv(file, index_col=0).to_numpy()
			elif file.split('.')[-1] == "tsp":
				self.matrix, self.coords = import_tsplib(file)
			 
		# On crée les villes dans le réseau et les ajoute à la liste.
		self.cities = []
		
		for i in np.arange(self.matrix.shape[0]):
			self.cities.append(City(self.matrix[i,:], i))
		
		# On marque la première ville dans le réseau. Par défaut est la première ville
		# ajoutée, mais on peut choisir.
		self.first = index_first
		self.current = index_first
		self.visit_order = [index_first]
		self.total_distance = 0
		self.visited = [c.visited for c in self.cities]
		self.visited[self.first] = True
		
	# Méthode pour ajouter les villes dans la liste ordonnée.
	def add_visited_city(self, index):
		self.visit_order.append(index)
		self.current = index


# Classe pour représenter une ville
# Membres: - visited : boolean pour marque si la ville est déjà visitée
#          - index : indice de la ville (indice de la file de la matrice de distance)
#          - neighbors : liste d'indices des voisins
#          - distances : distance vers les voisins
#
# Méthodes: - is_visited() : retourne self.visited
#           - mark_visited() : marque la ville comme visitée
class City():
	def __init__(self, row, index):
		self.visited = False
		self.index = index
		self.neighbors = np.nonzero(row)[0]
		self.distances = row[self.neighbors]
		
	def is_visited(self):
		return self.visited
	
	def mark_visited(self):
		self.visited = True

### Fonctions pour dessiner les résultats
bla

In [ ]:
# Fonction pour dessiner un graph
# Entrées: - coords : matrice de coordonnées des villes
#          - radius : taille du point dans le graph
#          - connected : décide si l'on ajoute les lignes
#          - embedded : décide si le graph est un image indépendant
#          - axis : axe à remplir si embedded est vrai
# Sorties: - l'image du graph
def plot_graph(coords, radius=10, connected=True, embedded=False, axis=None):
    # Si l'image est indépendant on cree la figure et ses axes
    if not embedded:
        fig, axis = plt.subplots()
    
    # On ajoute chaque ville au graph
    for i in range(coords.shape[0]):
        circle = plt.Circle((coords[i,0], coords[i,1]), radius, color='blue', fill=True)
        axis.add_patch(circle)
    
        # Si connecté, on ajoute les lignes entre villes
        if connected:
            for j in range(i + 1, coords.shape[0]):
                x_values = [coords[i,0], coords[j,0]]
                y_values = [coords[i,1], coords[j,1]]
                axis.plot(x_values, y_values, color='black', linestyle='-')
    
    # Si l'image est indépendant on choisit ses propriétés
    if not embedded:
        axis.set_aspect('equal', 'box')
        axis.autoscale()
        plt.grid(True)
        plt.show()


# Fonction pour dessiner un graph orienté
# Entrées: - coords : matrice de coordonnées des villes
#          - order : ordre de parcours des villes
#          - radius : taille du point dans le graph
#          - embedded : décide si le graph est un image indépendant
#          - axis : axe à remplir si embedded est vrai
# Sorties: - l'image du graph
def plot_directed_graph(coords, order, radius=10, embedded=False, axis=None):
    # Si l'image est indépendant on cree la figure et ses axes
    if not embedded:
        fig, axis = plt.subplots()
    
    # On ajoute chaque ville au graph. La première ville on ajoute en rouge
    for i in range(coords.shape[0]):
        if i == 0:
            circle = plt.Circle((coords[i,0], coords[i,1]), radius, color='red', fill=True)
        else:
            circle = plt.Circle((coords[i,0], coords[i,1]), radius, color='blue', fill=True)
        axis.add_patch(circle)
    
    # Pour chaque ville, on ajoute une flèche connectant le parcours
    for j in range(len(order)-1):
        x_values = [coords[order[j],0], coords[order[j+1],0]]
        y_values = [coords[order[j],1], coords[order[j+1],1]]
        axis.annotate("", xytext=(coords[order[j],0], coords[order[j],1]), xy=(coords[order[j+1],0], coords[order[j+1],1]), arrowprops=dict(arrowstyle="->"))
    
    # Si l'image est indépendant on choisit ses propriétés
    if not embedded:
        axis.set_aspect('equal', 'box')
        axis.autoscale()
        plt.grid(True)
        plt.show()

# Algorithme Voisins plus proches
Bla bla

## Implementation
nice algo

In [ ]:
# Algorithme des voisins plus proches
# Entrées: - map objet de réseau de villes
def nearest_neighbors(map):
    # On cree une variable auxiliaire que vérifie si tous les villes ont été visitées
	all_visited = False
 
	# Si l'on a pas visité tous les villes, on continue
	while not all_visited:
		# On choisit la ville actuelle
		curr_city = map.cities[map.current]
		best_distance = float('inf')
		best_neighbor_ind = None

		# On compare tous les voisins, chaque fois que la distance est inférieure, on
		# actualise le voisin
		for neighbor_index, distance in zip(curr_city.neighbors, curr_city.distances):
			if not map.visited[neighbor_index] and distance < best_distance:
				best_distance = distance
				best_neighbor_ind = neighbor_index

		# Après vérification de tous les voisins, on ajoute le voisin plus proche à
		# la liste et le marque comme visité. On met à jour l'indice actuel
		map.total_distance += best_distance
		map.visit_order.append(best_neighbor_ind)
		map.visited[best_neighbor_ind] = True
		map.cities[best_neighbor_ind].visited = True
		map.current = best_neighbor_ind
		
		# Si tous les villes sont visitées, on marque la variable comme vrai pour sortir
		# le boucle
		if all(map.visited):
			all_visited = True
   
	# Pour finir, on ajoute la première ville a la fin de la liste, pour fermer le parcours
	map.total_distance += map.matrix[map.current][map.first]
	map.visit_order.append(map.first)
	map.current = map.first

## Exemples
sfsd

# Algorithme 2-opt
blabla

## Implementation
nice algo

In [ ]:
# Fonction auxiliaire pour calculer la distance parcouru
# Entrées: - matrix : matrice de distance
#          - order : ordre de visite
# Sorties: - distance : distance parcouru
def compute_total_distance(matrix, order):
    distance = 0
    
    for i in range(len(order) - 1):
        distance += matrix[order[i], order[i+1]]
        
    return distance

# Algorithme 2-opt pour optimizer voisins proches
# Entrées: - map : objet de réseau de villes
def two_opt(map):
    # On considère l'état actuel comme la solution optimale
    best_distance = map.total_distance
    new_order = map.visit_order[:-1]
    n = len(new_order)
    improved = True

    # On cherche des séquences à optimiser
    while improved:
        improved = False
        
        # À partir de chaque noeud on prendre de chaînes
        for i in range(1, n - 1):
            # Pour chaque chaîne on fait le échange avec son inverse
            for j in range(i + 1, n):
                flipped_order = new_order[:i] + new_order[i:j+1][::-1] + new_order[j+1:]
                new_distance = compute_total_distance(map.matrix, flipped_order + [flipped_order[0]])

                # Si le chemin parcouru est moins avec le chemin inversé, on le mettre à jour
                if new_distance < best_distance:
                    new_order = flipped_order
                    best_distance = new_distance
                    improved = True
                    break
                
            if improved:
                break

    # Mis a jour du réseau
    map.visit_order = new_order + [new_order[0]]
    map.total_distance = best_distance

## Exemples
sdfs